In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, Ridge, Lasso, ElasticNet

import warnings
import itertools
import time

import cudf
import cupy as cp
from cuml.linear_model import Ridge, Lasso, ElasticNet

In [95]:
warnings.filterwarnings("ignore")

In [96]:
data_df = cudf.read_csv("../data/data.csv")
billboard_df = cudf.read_csv("../data/billboard/hot-100_all.csv")
pop_metrics = cudf.read_csv("../data/popularity_metrics.csv")

data_df.drop(labels='Unnamed: 0', axis=1, inplace=True)
pop_metrics.drop(labels='Unnamed: 0', axis=1, inplace=True)
pop_metrics['avg_rank_score'] = pop_metrics['avg_rank_score'].apply(lambda x: x*100)

In [97]:
type(data_df)

cudf.core.dataframe.DataFrame

In [98]:
billboard_df

,title,artist,image,peakPos,lastPos,weeks,rank,isNew,date
0,All I Want For Christmas Is You,Mariah Carey,<NA>,1,1,37,1,False,2020-01-04
1,Rockin' Around The Christmas Tree,Brenda Lee,<NA>,2,2,32,2,False,2020-01-04
2,Jingle Bell Rock,Bobby Helms,<NA>,3,9,30,3,False,2020-01-04
3,A Holly Jolly Christmas,Burl Ives,<NA>,4,6,15,4,False,2020-01-04
4,Circles,Post Malone,<NA>,1,3,17,5,False,2020-01-04
...,...,...,...,...,...,...,...,...,...
10495,Bubbly,Young Thug With Drake & Travis Scott,<NA>,20,100,10,96,False,2022-01-01
10496,Do It To It,Acraze Featuring Cherish,<NA>,97,0,1,97,True,2022-01-01
10497,No Love,Summer Walker & SZA,<NA>,13,96,7,98,False,2022-01-01
10498,Knowing You,Kenny Chesney,<NA>,57,84,19,99,False,2022-01-01


In [99]:
def gscv(mdl, param_grid, x_, y_, score_method, verbose=0):
    grid_search = GridSearchCV(mdl, param_grid=param_grid, n_jobs=-1, verbose=verbose, scoring=score_method)
    grid_search.fit(x_, y_)
    if verbose == 1:
        print(f"best params = {grid_search.best_params_}")
        print(f"best score = {grid_search.best_score_}")
    return grid_search.best_params_, grid_search.best_score_

In [100]:
def process_subset(model, X, y, features):
    kf = KFold()
    mse = []
    X = X[list(features)]
    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        regr = model.fit(X_train, y=y_train)
        y_predict = regr.predict(X_test)
        mse.append(mean_squared_error(y_test, y_predict))
    return {"model": model, "features": features, "MSE":cp.mean(mse)}

In [101]:
def best_subset(model, X, y, subset_size=-1, force=False):
    limit = 100000
    if subset_size == -1:
        subset_size = len(X.columns)
    assert len(X.columns) >= subset_size, f"X.columns must be >= subset_size. Given len(X.columns)={len(X.columns)}, subset_size={subset_size}"

    num_combinations = cp.math.factorial(len(X.columns))/(cp.math.factorial(len(X.columns)-subset_size) * cp.math.factorial(subset_size))
    if num_combinations > limit and not force:
        print(f"Please be aware that this action will run {int(num_combinations)} models and there is a real possibility it will crash your system.\nIf you're ABSOLUTELY SURE you want to do this please include the parameter 'force=True' when calling this function")
        return {"model": model, "features": X.columns, "MSE": -1}
    tic = time.time()
    results = []
    for combo in itertools.combinations(X.columns, subset_size):
        results.append(process_subset(model, X, y, combo))
    models = cudf.DataFrame(results)
    best_model = models.loc[models['MSE'].argmin()]
    toc = time.time()
    print("Processed", models.shape[0], "models on", subset_size, "predictors in", (toc-tic), "seconds.")
    return best_model

In [139]:
def stepwise_selection(model, X, y, direction='forward', num_features=None, scoring='neg_root_mean_squared_error'):
    sfs = SequentialFeatureSelector(model, direction=direction, n_features_to_select=num_features, scoring=scoring)
    X = X.to_numpy()
    y = y.to_numpy()
    sfs.fit(X, y)
    return sfs.transform(X), sfs.get_support()

In [140]:
# get n samples from 1-dimensional df following a gaussian distribution
def sample(df: cudf.DataFrame, n=400, plot=False):
    x = cp.sort(df)
    f_x = cp.gradient(x)*cp.exp(-x**2/2)
    sample_probs = f_x/cp.sum(f_x)
    df_samples = df.sort_values().sample(n=n, weights=sample_probs, replace=False)
    if plot:
        sns.distplot(df_samples)
    return df_samples

In [ ]:
def mask(item, mask):
    final = []
    for i in range(len(item)):
        if mask[i] == True:
            final.append(item[i])
    return final

In [ ]:
# sample(pop_metrics['avg_rank_score'], plot=True)
# features, idx_mask = stepwise_selection(Ridge(), data_df, pop_metrics['avg_rank_score'], num_features=2)
# fs = data_df.loc[:, idx_mask==True]
# fs = data_df.masked_assign(True, idx_mask)
cols = data_df.columns[0]
to_drop = []
for idx, i in enumerate(idx_mask):
    if i == True:
        to_drop.append(data_df.columns[idx])
data_df.drop(labels=to_drop, axis=1)

In [133]:
# features, idx_mask = stepwise_selection(LinearRegression(), data_df, pop_metrics['avg_rank_score'], direction='backward')
# bs = data_df.loc[:, idx_mask==True]

In [134]:
def run_regression(model, X, y):
    mse = []
    for train_idx, test_idx in KFold().split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        regr = model.fit(X_train, y=y_train)
        y_predict = regr.predict(X_test)
        mse.append(mean_squared_error(y_test, y_predict))
    return cp.mean(mse)

In [135]:
def run(mdl, X: cudf.DataFrame, y, forward=True, backward=True):
    tic = time.time()
    results = []
    if forward:
        for f in range(1, len(X.columns)):
            _, predictors_mask = stepwise_selection(mdl['model'], X, y, direction='forward', num_features=f)
            predictors = X.loc[:, predictors_mask==True]
            best_params, best_score = gscv(mdl['model'], mdl['cv'], predictors, y, score_method='neg_root_mean_squared_error', verbose=0)
            results.append({"model": mdl['model'], "best_parms": best_params, "best_score": best_score, "features": predictors.columns, "features_mask": predictors_mask, "feature_selection_method": "forward_selection", "num_features": f})

    if backward:
        for f in range(cp.arange(1, len(X.columns))):
            _, predictors_mask = stepwise_selection(mdl['model'], X, y, direction='backward', num_features=f)
            predictors = X.loc[:, predictors_mask==True]
            best_params, best_score = gscv(mdl['model'], mdl['cv'], predictors, y, score_method='neg_root_mean_squared_error', verbose=0)
            results.append({"model": mdl['model'], "best_parms": best_params, "best_score": best_score, "features": predictors.columns, "features_mask": predictors_mask, "feature_selection_method": "backward_selection", "num_features": f})

    best_params, best_score = gscv(mdl['model'], mdl['cv'], X, y, score_method='neg_root_mean_squared_error', verbose=0)
    results.append({"model": mdl['model'], "best_parms": best_params, "best_score": best_score, "features": X.columns, "features_mask": [], "feature_selection_method": "none", "num_features": -1})

    toc = time.time()

    print(f"Processed {len(results)} in {toc-tic} seconds.")
    return results

# y = sample(pop_metrics['avg_rank_score'], plot=False)
# X = data_df.iloc[y.index]
# x_train, x_test, y_train, y_test = train_test_split(X, y)
# mdl = RidgeCV(scoring='neg_root_mean_squared_error').fit(x_train, y_train)
# y_pred = mdl.predict(x_test)
# print(f"mse: {mean_squared_error(y_test, y_pred)}")
# fig = plt.figure(figsize=(10,5))
# plt.scatter(y_test,y_pred)
# plt.plot(y_test,y_test,'r', label='test')
# plt.plot(y_pred,y_pred,'b', label='pred')
# plt.show()
#
# # y = sample(pop_metrics['avg_rank_score'], plot=False)
# X = fs.iloc[y.index]
# x_train, x_test, y_train, y_test = train_test_split(X, y)
# mdl = RidgeCV(scoring='neg_root_mean_squared_error').fit(x_train, y_train)
# y_pred = mdl.predict(x_test)
# print(f"mse: {mean_squared_error(y_test, y_pred)}")
# fig = plt.figure(figsize=(10,5))
# plt.scatter(y_test,y_pred)
# plt.plot(y_test,y_test,'r', label='test')
# plt.plot(y_pred,y_pred,'b', label='pred')
#
# # y = sample(pop_metrics['avg_rank_score'], plot=False)
# X = bs.iloc[y.index]
# x_train, x_test, y_train, y_test = train_test_split(X, y)
# mdl = RidgeCV(scoring='neg_root_mean_squared_error').fit(x_train, y_train)
# y_pred = mdl.predict(x_test)
# print(f"mse: {mean_squared_error(y_test, y_pred)}")
# fig = plt.figure(figsize=(10,5))
# plt.scatter(y_test,y_pred)
# plt.plot(y_test,y_test,'r', label='test')
# plt.plot(y_pred,y_pred,'b', label='pred')



In [136]:
model_configs = {
    "models": [
        {"model": Ridge(), "cv": {}},
        {"model": Lasso(max_iter=10000), "cv": {}},
        {"model": ElasticNet(max_iter=10000), "cv": {}}
    ],
    "y": ['avg_rank_score'],
}

all_models = []
for m in model_configs['models']:
    for y in model_configs['y']:
        X_train, X_test, y_train, y_test = train_test_split(data_df, pop_metrics[y])
        all_models.append({
            "results": run(m, X_train, y_train, forward=True, backward=True), "X_test": X_test, "y_test": y_test
        })

ValueError: maximum supported dimension for an ndarray is 32, found 1067